In [85]:
import torch
import math

In [160]:
class tensor:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._prev = set(_children)
        self._op = _op
        self._backward = lambda: None

    def __add__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        out = tensor(self.data + other.data, _children=(self, other), _op='+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        out = tensor(self.data * other.data, _children=(self, other), _op='*')

        def _backward():
            self.grad += other.data*out.grad
            other.grad += self.data*out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        return self * other**(-1)

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def exp(self):
        x = self.data
        out = tensor(math.exp(x), _children=(self, ), _op='exp')

        def _backward():
            self.grad += out.data*out.grad
        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = tensor(t, _children=(self, ), _op='tanh')

        def _backward():
            self.grad += (1.0 - t**2)*out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            visited.add(v)
            for child in v._prev:
                build_topo(child)
            topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

    def item(self):
        return self.data

    def __repr__(self):
        return f"tensor({self.data})"


In [161]:
a = tensor(2.0)
b = tensor(-3.0)
c = tensor(4.0)

d = a*b + c + 3

In [162]:
(2+d).exp()

tensor(20.085536923187668)

In [123]:
d.backward()
print (a.grad)
print (b.grad)
print (c.grad)
print (d.grad)
 

-3.0
2.0
1.0
1.0


In [115]:
topo = []
visited = set()
def build_topo(v):
    visited.add(v)
    for child in v._prev:
        build_topo(child)
    topo.append(v)
build_topo(d)
topo

[tensor(40.0), tensor(2.0), tensor(-3.0), tensor(-6.0), tensor(34.0)]

In [54]:
a = tensor(2.0)
b = tensor(-3.0)
c = tensor(4.0)
print (a+b)
print (a*b + c)
print ((a*b + c).item())

tensor(-1.0)
tensor(-2.0)
-2.0


In [56]:
x = tensor (a*b + c)

x._prev

set()

In [44]:
a = tensor([2.0])
b = tensor([3.0])


In [45]:
a + b

tensor([2.0, 3.0])

In [139]:
a = tensor(2.0)
b = tensor(-3.0)
c = tensor(4.0)

d = a*b + c
e = d.tanh()

e.backward()
print (a.grad)
print (b.grad)
print (c.grad)
print (d.grad)


-0.21195247455949395
0.1413016497063293
0.07065082485316465
0.07065082485316465


In [138]:
print (e)

tensor(-0.9640275800758168)


In [136]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(-3.0, requires_grad=True)
c = torch.tensor(4.0, requires_grad=True)

d = a*b + c
e = d.tanh()

e.backward()
print (a.grad)
print (b.grad)
print (c.grad)
print (d.grad)

tensor(-0.2120)
tensor(0.1413)
tensor(0.0707)
None


/var/folders/g3/s8dw43sn4cbb99pp7s3zn8vc0008fx/T/ipykernel_71562/2217365474.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:485.)
  print (d.grad)
